In [1]:
from datetime import datetime

from sqlalchemy import(MetaData, Table, Column, Integer, Numeric, String,
                      DateTime, ForeignKey, Boolean, create_engine, CheckConstraint)

metadata = MetaData()

cookies = Table('cookies', metadata,
    Column('cookie_id', Integer(), primary_key=True),
    Column('cookie_name', String(55), index=True),
    Column('cookie_recipe_url', String(255)),
    Column('cookie_sku', String(55)),
    Column('quantity', Integer()),
    Column('unit_cost', Numeric(12,2)),
    CheckConstraint('quantity >= 0', name='quantity_positive')
)

users = Table('users', metadata,
    Column('user_id', Integer(), primary_key=True),
    Column('username', String(15), nullable=False, unique=True),
    Column('email_address', String(255), nullable=False),
    Column('phone', String(20), nullable=False),
    Column('password', String(25), nullable=False),
    Column('create_on', DateTime(), default=datetime.now),
    Column('updated_on', DateTime(), default=datetime.now, onupdate=datetime.now),
)

orders = Table('orders', metadata,
    Column('order_id', Integer(), primary_key= True),
    Column('user_id', ForeignKey('users.user_id')),
    Column('shipped', Boolean(), default=False)
)
line_items = Table('line_items', metadata,
    Column('line_items_id', Integer(), primary_key=True),
    Column('order_id', ForeignKey('orders.order_id')),
    Column('cookie_id', ForeignKey('cookies.cookie_id')),
    Column('quantity', Integer()),
    Column('exetended_cost', Numeric(12, 2))
)

engine=create_engine('sqlite:///:memory:')
metadata.create_all(engine)
connection = engine.connect()

In [2]:
from sqlalchemy import select, insert, update
ins = insert(users).values(
    username="cookiemon",
    email_address="mon@cookie.com",
    phone="111-111-1111",
    password="password"
)
result = connection.execute(ins)

In [3]:
ins=cookies.insert()
inventory_list = [
    {
        'cookie_name': 'chocolate chip',
        'cookie_recipe_url': 'http://some.aweso.me/cookie/recipe.html',
        'cookie_sku': 'CC01',
        'quantity': '12',
        'unit_cost': '0.50'
    },
     {
        'cookie_name': 'dark chocolate chip',
        'cookie_recipe_url': 'http://some.aweso.me/cookie/recipe_dark.html',
        'cookie_sku': 'CC02',
        'quantity': '1',
        'unit_cost': '0.75'
    }
]
result = connection.execute(ins, inventory_list)

In [4]:
ins = insert(orders).values(user_id=1, order_id=1)
result=connection.execute(ins)
ins = insert(line_items)
order_items = [
    {
        'order_id': 1,
        'cookie_id': 1,
        'quantity': 9,
        'extended_cost':4.50
    }
]
result=connection.execute(ins, order_items)

In [5]:
ins = insert(orders).values(user_id=1, order_id=2)
result=connection.execute(ins)
ins = insert(line_items)
order_items = [
    {
        'order_id': 2,
        'cookie_id': 2,
        'quantity': 1,
        'extended_cost':1.50
    },
        {
        'order_id': 2,
        'cookie_id': 1,
        'quantity': 4,
        'extended_cost':4.50
    }
]
result=connection.execute(ins, order_items)

In [6]:
def ship_it(order_id):
    
    s=select([line_items.c.cookie_id, line_items.c.quantity])
    s =s.where(line_items.c.order_id==order_id)
    cookies_to_ship = connection.execute(s)
    for cookie in cookies_to_ship:
        u = update(cookies).where(cookies.c.cookie_id==cookie.cookie_id)
        u = u.values(quantity = cookies.c.quantity - cookie.quantity)
        result = connection.execute(u)
    u =update(orders).where(orders.c.order_id==order_id)
    u = u.values(shipped=True)
    result = connection.execute(u)
    print("Shipped order ID: {}".format(order_id))
    

In [7]:
ship_it(1)

Shipped order ID: 1


In [8]:
s = select([cookies.c.cookie_name, cookies.c.quantity])
connection.execute(s).fetchall()

[('chocolate chip', 3), ('dark chocolate chip', 1)]

In [9]:
ship_it(2)

IntegrityError: (sqlite3.IntegrityError) CHECK constraint failed: quantity_positive [SQL: 'UPDATE cookies SET quantity=(cookies.quantity - ?) WHERE cookies.cookie_id = ?'] [parameters: (4, 1)] (Background on this error at: http://sqlalche.me/e/gkpj)

In [ ]:
s = select([cookies.c.cookie_name, cookies.c.quantity])
connection.execute(s).fetchall()

In [10]:
print("Critical Thinking Section:  In your own words, write a paragraph (5 sentences or more) describing how you would save your data from Python without using a database?  In other words, what are alternative means you can use to store your Python information?")
print("Answer: ")
print("I can store the data using SQLite. SQLite isnt a client-server database but relational database. You can also store data in an array. Using multiple arrays for multiple inserts.")

Critical Thinking Section:  In your own words, write a paragraph (5 sentences or more) describing how you would save your data from Python without using a database?  In other words, what are alternative means you can use to store your Python information?
Answer: 
I can store the data using SQLite. SQLite isnt a client-server database but relational database. You can also store data in an array. Using multiple arrays for multiple inserts.


In [ ]:
print("Mathew Holden")